# 70/30 split

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd

# Import relevant scikit-learn modules
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_validate

import ModelFits

In [2]:
# read .csv format data
Data_path = '../Data/data_160_ori.csv'
REPR_DIR = '../Reprs'

repr_path= []
for file in os.listdir(REPR_DIR):
    if '.csv' in file:
        path = REPR_DIR+ '/' + file
        repr_path.append(path)

repr_path

['../Reprs/Solvent_Repr.csv',
 '../Reprs/Solvent_Repr_160.csv',
 '../Reprs/Solvent_Repr_180.csv',
 '../Reprs/sol_oh.csv',
 '../Reprs/sol_oh_180.csv',
 '../Reprs/UniMolRepr.csv',
 '../Reprs/UniMolRepr_160.csv',
 '../Reprs/UniMolRepr_180.csv']

In [3]:
df = pd.read_csv(Data_path, index_col=0)
unimol_repr = pd.read_csv(repr_path[-2],index_col=0)
sol_repr = pd.read_csv(repr_path[1],index_col=0)
sol_oh = pd.read_csv(repr_path[3],index_col=0)
# sol_oh = sol_oh[0:160]

# concat required features
fea_df = pd.concat([unimol_repr,sol_repr], axis=1)

# Extract label colunm
label_df = df['yields_label']

In [4]:
# test yields
# label Encoding
le = LabelEncoder()
label_le = le.fit_transform(label_df) #  0: high 1: low, 2: medium


In [5]:
ee = df['ee'].copy()

low = []
med = []
high = []

for i in range(len(ee)):
    if ee[i] >= 80:
        high.append(i)
        ee.iloc[i] = 'high'
    elif ee[i] <= 20:
        low.append(i)
        ee.iloc[i] = 'low'
    else:
        med.append(i)
        ee.iloc[i] = 'medium'
label_df2 = ee

le2 = LabelEncoder()
label_le2 = le2.fit_transform(label_df2) # medium 2, low 1

C:\Users\Administrator\AppData\Local\Temp\ipykernel_26880\3273759680.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'medium' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ee.iloc[i] = 'medium'


In [14]:
X = fea_df
y = label_le
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier on the resampled training data
rf_classifier.fit(X_train, y_train)

# Predictions on the original test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[ 3  1  0]
 [ 0 37  1]
 [ 0  2  4]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.93      0.97      0.95        38
           2       0.80      0.67      0.73         6

    accuracy                           0.92        48
   macro avg       0.91      0.80      0.84        48
weighted avg       0.92      0.92      0.91        48



In [16]:
for i in range(40, 50):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
    class_weights = {0: 3, 1: 1, 2: 2} # O:High More important; 1: Medium Less important; 2: Low last important
    rf_classifier = RandomForestClassifier(random_state=i, class_weight=class_weights) # class_weight=class_weights

    # Train the classifier on the sampled training data
    rf_classifier.fit(X_train, y_train)

    # Predictions on the original test set
    y_pred = rf_classifier.predict(X_test)

    # Evaluate the model
    print(f'*********************Random State: {i}**********************')
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print('-------------------------------------------------------------')

*********************Random State: 40**********************
Confusion Matrix:
 [[ 4  2  1]
 [ 0 29  0]
 [ 1  7  4]]

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.76      1.00      0.87        29
           2       0.80      0.33      0.47        12

    accuracy                           0.77        48
   macro avg       0.79      0.63      0.67        48
weighted avg       0.78      0.77      0.74        48

-------------------------------------------------------------
*********************Random State: 41**********************
Confusion Matrix:
 [[ 3  2  1]
 [ 0 30  3]
 [ 2  5  2]]

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.81      0.91      0.86        33
           2       0.33      0.22      0.27         9

    accuracy                           0.73        48
   

In [32]:
for i in range(40, 50):

    X_train, X_test, y_train, y_test = train_test_split(X, label_le2, test_size=0.3, random_state=i)
    class_weights = {0: 3, 1: 1, 2: 2} # O:High More important; 1: Medium Less important; 2: Low last important
    rf_classifier = RandomForestClassifier(random_state=i, class_weight=class_weights) # class_weight=class_weights

    # Train the classifier on the sampled training data
    rf_classifier.fit(X_train, y_train)

    # Predictions on the original test set
    y_pred = rf_classifier.predict(X_test)

    # Evaluate the model
    print(f'*********************Random State: {i}**********************')
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print('-------------------------------------------------------------')

*********************Random State: 40**********************
Confusion Matrix:
 [[ 3  0  2]
 [ 1  9  4]
 [ 0  2 27]]

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.82      0.64      0.72        14
           2       0.82      0.93      0.87        29

    accuracy                           0.81        48
   macro avg       0.80      0.72      0.75        48
weighted avg       0.81      0.81      0.81        48

-------------------------------------------------------------
*********************Random State: 41**********************
Confusion Matrix:
 [[ 0  0  0]
 [ 1  8  4]
 [ 5  2 28]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.80      0.62      0.70        13
           2       0.88      0.80      0.84        35

    accuracy                           0.75        48
   

d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


*********************Random State: 42**********************
Confusion Matrix:
 [[ 2  0  2]
 [ 0  8  8]
 [ 0  3 25]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.73      0.50      0.59        16
           2       0.71      0.89      0.79        28

    accuracy                           0.73        48
   macro avg       0.81      0.63      0.68        48
weighted avg       0.74      0.73      0.72        48

-------------------------------------------------------------
*********************Random State: 43**********************
Confusion Matrix:
 [[ 1  1  1]
 [ 0  9  5]
 [ 2  9 20]]

Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.47      0.64      0.55        14
           2       0.77      0.65      0.70        31

    accuracy                           0.62        48
   

d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 分层抽样 交叉验证 

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# model = RandomForestClassifier(
#     n_estimators=100,         # 树的数量
#     max_depth=10,             # 树的最大深度，防止过拟合
#     min_samples_split=5,      # 内部节点再划分所需的最小样本数
#     min_samples_leaf=4,       # 叶节点所需的最小样本数
#     class_weight='balanced',  # 处理类别不均衡
#     random_state=42)

model = RandomForestClassifier(
    n_estimators=93,         # 树的数量
    max_depth=19,             # 树的最大深度，防止过拟合
    min_samples_split=4,      # 内部节点再划分所需的最小样本数
    min_samples_leaf=4,       # 叶节点所需的最小样本数
    class_weight= {0: 4, 1: 1, 2: 2},  # 处理类别不均衡
    max_features='sqrt',
    random_state=42)



# 使用多个评分标准
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']


X = fea_df
y = label_le2

# 执行交叉验证，获取各类评分
cv_results = cross_validate(model, X, y, cv=kf, scoring=scoring, return_train_score=True)

# 输出每次划分的训练集和测试集评分
print("每次划分的训练集评分:")
for i in range(10):
    print(f"第{i+1}次训练集准确率: {cv_results['train_accuracy'][i]:.4f}")
    print(f"第{i+1}次训练集精确度: {cv_results['train_precision_macro'][i]:.4f}")
    print(f"第{i+1}次训练集召回率: {cv_results['train_recall_macro'][i]:.4f}")
    print(f"第{i+1}次训练集F1分数: {cv_results['train_f1_macro'][i]:.4f}")

print("\n每次划分的测试集评分:")
for i in range(10):
    print(f"第{i+1}次测试集准确率: {cv_results['test_accuracy'][i]:.4f}")
    print(f"第{i+1}次测试集精确度: {cv_results['test_precision_macro'][i]:.4f}")
    print(f"第{i+1}次测试集召回率: {cv_results['test_recall_macro'][i]:.4f}")
    print(f"第{i+1}次测试集F1分数: {cv_results['test_f1_macro'][i]:.4f}")

# 输出平均评分
print("\n平均评分:")
print(f"平均训练集准确率: {cv_results['train_accuracy'].mean():.4f}")
print(f"平均训练集精确度: {cv_results['train_precision_macro'].mean():.4f}")
print(f"平均训练集召回率: {cv_results['train_recall_macro'].mean():.4f}")
print(f"平均训练集F1分数: {cv_results['train_f1_macro'].mean():.4f}")

print(f"平均测试集准确率: {cv_results['test_accuracy'].mean():.4f}")
print(f"平均测试集精确度: {cv_results['test_precision_macro'].mean():.4f}")
print(f"平均测试集召回率: {cv_results['test_recall_macro'].mean():.4f}")
print(f"平均测试集F1分数: {cv_results['test_f1_macro'].mean():.4f}")

d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


每次划分的训练集评分:
第1次训练集准确率: 0.9514
第1次训练集精确度: 0.9178
第1次训练集召回率: 0.9683
第1次训练集F1分数: 0.9410
第2次训练集准确率: 0.9028
第2次训练集精确度: 0.8470
第2次训练集召回率: 0.8930
第2次训练集F1分数: 0.8676
第3次训练集准确率: 0.9444
第3次训练集精确度: 0.9035
第3次训练集召回率: 0.9390
第3次训练集F1分数: 0.9189
第4次训练集准确率: 0.9306
第4次训练集精确度: 0.8837
第4次训练集召回率: 0.9422
第4次训练集F1分数: 0.9099
第5次训练集准确率: 0.9514
第5次训练集精确度: 0.9344
第5次训练集召回率: 0.9359
第5次训练集F1分数: 0.9322
第6次训练集准确率: 0.9444
第6次训练集精确度: 0.9161
第6次训练集召回率: 0.9489
第6次训练集F1分数: 0.9312
第7次训练集准确率: 0.9306
第7次训练集精确度: 0.8770
第7次训练集召回率: 0.9315
第7次训练集F1分数: 0.9016
第8次训练集准确率: 0.9583
第8次训练集精确度: 0.9191
第8次训练集召回率: 0.9718
第8次训练集F1分数: 0.9433
第9次训练集准确率: 0.9167
第9次训练集精确度: 0.8602
第9次训练集召回率: 0.9355
第9次训练集F1分数: 0.8926
第10次训练集准确率: 0.9306
第10次训练集精确度: 0.8808
第10次训练集召回率: 0.9259
第10次训练集F1分数: 0.8965

每次划分的测试集评分:
第1次测试集准确率: 0.7500
第1次测试集精确度: 0.4167
第1次测试集召回率: 0.4167
第1次测试集F1分数: 0.4167
第2次测试集准确率: 0.8125
第2次测试集精确度: 0.5278
第2次测试集召回率: 0.5000
第2次测试集F1分数: 0.5093
第3次测试集准确率: 0.6875
第3次测试集精确度: 0.7333
第3次测试集召回率: 0.7424
第3次测试集F1分数: 0.7354
第4次测试集准确率: 0.8750
第4次测

In [38]:
#超参数优化

import optuna
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, label_le2, test_size=0.3, random_state=i)

def objective(trial):
    # 超参数空间定义
    n_estimators = trial.suggest_int('n_estimators', 50, 200)  # 树的数量
    max_depth = trial.suggest_int('max_depth', 3, 20)  # 树的最大深度
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)  # 内部节点再划分所需的最小样本数
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)  # 叶节点所需的最小样本数
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None]) # 特征选择方式

    # 创建随机森林分类器
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 预测并计算准确率
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy  # 返回优化目标（这里是准确率）

# 创建 Optuna 学习任务
study = optuna.create_study(direction='maximize')  # 最大化目标函数（准确率）
study.optimize(objective, n_trials=100)  # 进行 100 次超参数优化

# 输出最优的超参数和对应的结果
print(f"Best hyperparameters: {study.best_params}")
print(f"Best accuracy: {study.best_value:.4f}")

[I 2025-01-17 19:48:27,022] A new study created in memory with name: no-name-ea830269-8e7d-4beb-8518-cede42cc3624
[I 2025-01-17 19:48:27,167] Trial 0 finished with value: 0.7291666666666666 and parameters: {'n_estimators': 155, 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 0 with value: 0.7291666666666666.
[I 2025-01-17 19:48:29,172] Trial 1 finished with value: 0.7291666666666666 and parameters: {'n_estimators': 106, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 0 with value: 0.7291666666666666.
[I 2025-01-17 19:48:29,246] Trial 2 finished with value: 0.75 and parameters: {'n_estimators': 77, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 2 with value: 0.75.
[I 2025-01-17 19:48:29,368] Trial 3 finished with value: 0.75 and parameters: {'n_estimators': 88, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 5, 'm

Best hyperparameters: {'n_estimators': 93, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'sqrt'}
Best accuracy: 0.7708


In [74]:
# Try on MLP
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

X = fea_df

def objective(trial):
    # 定义优化的超参数空间
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(100,), (100, 50), (150, 100)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    solver = trial.suggest_categorical('solver', ['adam', 'sgd', 'lbfgs'])
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)  # 正则化参数，取对数分布
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive'])
    max_iter = trial.suggest_int('max_iter', 200, 500)

    # 创建 MLP 模型
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                            activation=activation,
                            solver=solver,
                            alpha=alpha,
                            learning_rate=learning_rate,
                            max_iter=max_iter,
                            random_state=42)
    
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    # 进行分层交叉验证
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # 对特征进行标准化处理
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)  # 训练集标准化
        X_test_scaled = scaler.transform(X_test)        # 测试集标准化

        # 训练模型
        model.fit(X_train_scaled, y_train)

        # 预测
        y_pred = model.predict(X_test_scaled)

        # 计算各项评分
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

        # 存储评分
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # 计算平均得分，返回优化目标（可以选择不同的得分）
    mean_accuracy = np.mean(accuracies)
    return mean_accuracy  # Optuna 会最大化这个目标值

study = optuna.create_study(direction='maximize')  # 最大化目标函数
study.optimize(objective, n_trials=50)  # 进行50次优化实验

# 输出优化结果
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: {trial.params}")

# 使用最优的超参数训练模型
best_model = MLPClassifier(
    hidden_layer_sizes=trial.params['hidden_layer_sizes'],
    activation=trial.params['activation'],
    solver=trial.params['solver'],
    alpha=trial.params['alpha'],
    learning_rate=trial.params['learning_rate'],
    max_iter=trial.params['max_iter'],
    random_state=42
)
# 使用多个评分标准
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

X = fea_df
y = label_le

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 执行交叉验证，获取各类评分
cv_results = cross_validate(model, X, y, cv=kf, scoring=scoring, return_train_score=True)

# 输出每次划分的训练集和测试集评分
print("每次划分的训练集评分:")
for i in range(10):
    print(f"第{i+1}次训练集准确率: {cv_results['train_accuracy'][i]:.4f}")
    print(f"第{i+1}次训练集精确度: {cv_results['train_precision_macro'][i]:.4f}")
    print(f"第{i+1}次训练集召回率: {cv_results['train_recall_macro'][i]:.4f}")
    print(f"第{i+1}次训练集F1分数: {cv_results['train_f1_macro'][i]:.4f}")

print("\n每次划分的测试集评分:")
for i in range(10):
    print(f"第{i+1}次测试集准确率: {cv_results['test_accuracy'][i]:.4f}")
    print(f"第{i+1}次测试集精确度: {cv_results['test_precision_macro'][i]:.4f}")
    print(f"第{i+1}次测试集召回率: {cv_results['test_recall_macro'][i]:.4f}")
    print(f"第{i+1}次测试集F1分数: {cv_results['test_f1_macro'][i]:.4f}")

# 输出平均评分
print("\n平均评分:")
print(f"平均训练集准确率: {cv_results['train_accuracy'].mean():.4f}")
print(f"平均训练集精确度: {cv_results['train_precision_macro'].mean():.4f}")
print(f"平均训练集召回率: {cv_results['train_recall_macro'].mean():.4f}")
print(f"平均训练集F1分数: {cv_results['train_f1_macro'].mean():.4f}")

print(f"平均测试集准确率: {cv_results['test_accuracy'].mean():.4f}")
print(f"平均测试集精确度: {cv_results['test_precision_macro'].mean():.4f}")
print(f"平均测试集召回率: {cv_results['test_recall_macro'].mean():.4f}")
print(f"平均测试集F1分数: {cv_results['test_f1_macro'].mean():.4f}")

[I 2025-01-18 10:50:49,305] A new study created in memory with name: no-name-39f1db3a-e339-4cd5-9b6e-9dbfbbb07543
d:\anaconda\envs\unimol_tools\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
d:\anaconda\envs\unimol_tools\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 50) which is of type tuple.
  warnings.warn(message)
d:\anaconda\envs\unimol_tools\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (150, 100) which is of type tuple.
  warnings.warn(message)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_25904\16

Best trial:
  Value: 0.84375
  Params: {'hidden_layer_sizes': (100, 50), 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.014822359495937095, 'learning_rate': 'invscaling', 'max_iter': 211}


d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maxi

每次划分的训练集评分:
第1次训练集准确率: 0.9931
第1次训练集精确度: 0.9967
第1次训练集召回率: 0.9885
第1次训练集F1分数: 0.9925
第2次训练集准确率: 0.9861
第2次训练集精确度: 0.9848
第2次训练集召回率: 0.9848
第2次训练集F1分数: 0.9848
第3次训练集准确率: 0.9653
第3次训练集精确度: 0.9588
第3次训练集召回率: 0.9662
第3次训练集F1分数: 0.9624
第4次训练集准确率: 0.6944
第4次训练集精确度: 0.2315
第4次训练集召回率: 0.3333
第4次训练集F1分数: 0.2732
第5次训练集准确率: 0.6944
第5次训练集精确度: 0.2315
第5次训练集召回率: 0.3333
第5次训练集F1分数: 0.2732
第6次训练集准确率: 0.6944
第6次训练集精确度: 0.2315
第6次训练集召回率: 0.3333
第6次训练集F1分数: 0.2732
第7次训练集准确率: 0.6944
第7次训练集精确度: 0.2315
第7次训练集召回率: 0.3333
第7次训练集F1分数: 0.2732
第8次训练集准确率: 0.6944
第8次训练集精确度: 0.2315
第8次训练集召回率: 0.3333
第8次训练集F1分数: 0.2732
第9次训练集准确率: 0.6944
第9次训练集精确度: 0.2315
第9次训练集召回率: 0.3333
第9次训练集F1分数: 0.2732
第10次训练集准确率: 0.6944
第10次训练集精确度: 0.2315
第10次训练集召回率: 0.3333
第10次训练集F1分数: 0.2732

每次划分的测试集评分:
第1次测试集准确率: 0.7500
第1次测试集精确度: 0.3932
第1次测试集召回率: 0.4167
第1次测试集F1分数: 0.4044
第2次测试集准确率: 0.7500
第2次测试集精确度: 0.4786
第2次测试集召回率: 0.4697
第2次测试集F1分数: 0.4683
第3次测试集准确率: 0.6875
第3次测试集精确度: 0.7333
第3次测试集召回率: 0.7424
第3次测试集F1分数: 0.7354
第4次测试集准确率: 0.6875
第4次测

d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\metrics\_classification

# 分组交叉验证， each catalyst as a group

In [68]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

X = fea_df  
y = label_le
groups = np.repeat(np.arange(32), 5)  # 每5条数据是一个组，总共有36组

# 创建随机森林模型
model = RandomForestClassifier(
    n_estimators=93,         # 树的数量
    max_depth=19,             # 树的最大深度，防止过拟合
    min_samples_split=4,      # 内部节点再划分所需的最小样本数
    min_samples_leaf=4,       # 叶节点所需的最小样本数
    class_weight= {0: 4, 1: 1, 2: 2},  # 处理类别不均衡
    max_features='sqrt',
    random_state=42)

# 创建 GroupKFold 对象
gkf = GroupKFold(n_splits=16)

# 存储每次测试集的评分
accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []

# 进行分组交叉验证
for train_idx, test_idx in gkf.split(X, y, groups=groups):
    # print("Train indices:", train_idx)
    # print("Test indices:", test_idx)
    
    # 使用 .iloc 基于整数位置索引划分数据集
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # 训练模型
    model.fit(X_train, y_train)

    # 预测
    y_pred = model.predict(X_test)
    
    # 计算各项评分
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    confusion = confusion_matrix(y_test, y_pred)


    # 存储评分
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    confusion_matrices.append(confusion)

# 输出每次交叉验证的准确率以及其他评分
print(f"Accuracies: {accuracies}")
print(f"Mean Accuracy: {np.mean(accuracies):.4f}")

print(f"Precisions: {precisions}")
print(f"Mean Precision: {np.mean(precisions):.4f}")

print(f"Recalls: {recalls}")
print(f"Mean Recall: {np.mean(recalls):.4f}")

print(f"F1 Scores: {f1_scores}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")

# 输出混淆矩阵（每个折的混淆矩阵）
print(f"Confusion Matrices: {confusion_matrices}")



Accuracies: [0.5714285714285714, 0.6571428571428571, 0.8333333333333334, 0.8, 0.6]
Mean Accuracy: 0.6924
Precisions: [0.19047619047619047, 0.31989247311827956, 0.2777777777777778, 0.26666666666666666, 0.3209876543209877]
Mean Precision: 0.2752
Recalls: [0.3333333333333333, 0.40444444444444444, 0.3333333333333333, 0.3333333333333333, 0.3666666666666667]
Mean Recall: 0.3542
F1 Scores: [0.24242424242424243, 0.35714285714285715, 0.30303030303030304, 0.29629629629629634, 0.30885780885780884]
Mean F1 Score: 0.3016
Confusion Matrices: [array([[ 0,  5,  0],
       [ 0, 20,  0],
       [ 0, 10,  0]], dtype=int64), array([[ 0,  7,  0],
       [ 0, 22,  3],
       [ 0,  2,  1]], dtype=int64), array([[ 0,  1,  0],
       [ 0, 25,  0],
       [ 0,  4,  0]], dtype=int64), array([[ 0,  1,  0],
       [ 0, 24,  0],
       [ 0,  5,  0]], dtype=int64), array([[ 0,  1,  2],
       [ 0, 17,  0],
       [ 0,  9,  1]], dtype=int64)]
